In [1]:
import sys, os

#temporary 
sys.path.append("../../../learnMSA")

# os.environ["CUDA_VISIBLE_DEVICES"] = ""
sys.path.append("../../bin")
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import tensorflow.keras as keras
from eval_model_class import PredictionGTF
from models import lstm_model

2024-10-15 11:49:12.967451: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-15 11:49:12.981065: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-15 11:49:12.985139: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-15 11:49:12.995625: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-15 11:49:13.534143: W tensorflow/compiler/tf2

In [3]:
batch_size = 2
seq_len = 500004
strand = '+'

emb = False
hmm_factor = 1
hmm_parallel = 817
trans_lstm = False

inp_data_dir = 'inp/'
out_dir = 'test_train/'
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

model_path = "../../model_weights/tiberius_weights"
genome_path = f'{inp_data_dir}/genome.fa'
# output gtf file
gtf_out = 'tiberius.out'

pred_gtf = PredictionGTF( 
    model_path=model_path,
     seq_len=seq_len, 
    batch_size=batch_size,
    hmm=True, 
    emb=False, 
    num_hmm=1,
    hmm_factor=1,
    genome_path=genome_path,
    softmask=True, strand=strand,
    parallel_factor=hmm_parallel
)

In [4]:
# load model
pred_gtf.load_model()

# load input data x_seq 
x_seq, y_seq, coords = pred_gtf.load_inp_data(    
    strand=strand, 
    chunk_coords=True, softmask=True
)

ValueError: File format not supported: filepath=../../model_weights/tiberius_weights. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(../../model_weights/tiberius_weights, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).

In [4]:
# generate LSTM and HMM predictions
hmm_pred = pred_gtf.get_predictions(x_seq, hmm_filter=True)

# infer gene structures and write GTF file
pred_gtf.create_gtf(y_label=hmm_pred, coords=coords,
        out_file=gtf_out, f_chunks=x_seq, strand=strand)

### LSTM prediction


2024-10-15 11:37:05.609978: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8800
2024-10-15 11:37:06.077531: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-15 11:37:06.078199: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-15 11:37:06.078212: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-10-15 11:37:06.078903: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-15 11:37:06.078949: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


LSTM took 1.8392102360725402 minutes to execute.
### HMM Viterbi
HMM took 0.5478992462158203 minutes to execute.
### LSTM prediction
LSTM took 0.28611326615015664 minutes to execute.
### HMM Viterbi
HMM took 0.05078897476196289 minutes to execute.


(<genome_anno.Anno at 0x7f6a70451de0>, 76)